In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import os
import numpy as np
import pandas as pd
import time
import torch
import pickle
import shap
import tensorboard


from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from functools import partial
import shutil
from tqdm.auto import tqdm

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

from codes.models.data_form.DataForm import DataTransfo_1SNP
from codes.models.metrics import calculate_roc_auc

import featurewiz as gwiz
from sklearn.cluster import KMeans


import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from codes.models.Decision_tree.utils import get_indice, get_name



In [ ]:
from itertools import combinations

import networkx as nx
from sklearn.metrics import f1_score

from pgmpy.estimators import PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling

In [ ]:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Abby' # Paul, Abby
rollup_depth = 4
Classes_nb = 2 #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
ld = 'no'
padding_token = 0
prop_train_test = 0.8
load_data = True
save_data = True
remove_none = False
decorelate = False
equalize_label = False
threshold_corr = 0.8
threshold_rare = 1000
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = True
list_env_features = []
list_phenos_ids = None#list(np.load('/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/phewas/list_associations_snps/rs673604_paul.npy'))
### data format
batch_size = 20
data_share = 1

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=remove_none,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share,
                         list_pheno_ids=list_phenos_ids,
                         ld=ld)
#patient_list = dataT.get_patientlist()''


In [ ]:
data, labels, indices_env, env_features, eids = dataT.get_tree_data(with_env=False, with_counts=False, load_possible=True, only_relevant=True)

In [ ]:
nb_keep_x, nb_phenos_keep = 100000, 100
data_try = data[:nb_keep_x:, :nb_phenos_keep]
columns = get_name(dataT, np.arange(nb_phenos_keep))
df = pd.DataFrame(data = data_try, columns=columns)

In [ ]:
# Initialize the PC algorithm object
pc = PC(data=df)

# Run the PC algorithm to learn the structure
estimated_model = pc.estimate(variant='stable', max_cond_vars=3)

In [ ]:
print(estimated_model.edges())


In [ ]:
data = get_example_model('alarm')


In [ ]:
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_array(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_array(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)

In [ ]:
tf.where(r, 0.8, 0.2)

In [ ]:
tfd.Bernoulli(probs=tf.where(r, 0.8, 0.2)).sample()